<a href="https://colab.research.google.com/github/makingthefuturehappy/bonus/blob/main/bonus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H1>Bonus</H1>

Drivers' segments description
https://docs.google.com/spreadsheets/d/11KIaZaywoBq3MymrCd8dmWJdBhFO989SNsrrSKYiB3Y/edit#gid=0

**Настраиваем переменные расчета**

In [ ]:
path = "bogota_bonus.csv"
cohorts = 5           # quantity of cohorts
commission = 0.095    # indriver commission
win_rate = 0.15       # percent of drivers who moves to the upper cohort
period_duration = 0.5 # period to forecast prospective rides from the total period

## Загружаем данные и анализируем кол-во поездок для каждого водителя

In [ ]:
#
import pandas as pd
import numpy as np
import math
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# percentile level calc
percentile_level = {}
percentile = 0
for i in range (1, cohorts):
    percentile = 100 / cohorts + percentile
    percentile_level[i] = int(percentile)

<h3>Data upload and pre-calc</h3>

In [ ]:
df = pd.read_csv(path)
df['gmv'] = pd.to_numeric(df['gmv'], errors='coerce')
dates = list(df.columns)[2:]
print("total drivers:", df.shape[0])
df.sample(n=5)

total drivers: 25123


,id,gmv,29.05.22,30.05.22,31.05.22,01.06.22,02.06.22,03.06.22,04.06.22,05.06.22,...,17.06.22,18.06.22,19.06.22,20.06.22,21.06.22,22.06.22,23.06.22,24.06.22,25.06.22,26.06.22
20988,117494312,7.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16994,123317001,31.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7863,66409377,50.0,0,0,0,0,1,1,1,2,...,0,0,0,1,0,2,0,1,0,1
95,48628238,141.0,0,0,0,0,0,0,2,5,...,0,5,5,0,0,0,0,2,7,3
4428,53882837,105.0,1,0,0,3,1,2,1,0,...,0,0,0,0,2,1,0,2,1,0


<h3>retention & avr. check calc</h3>

In [ ]:
# date split by weekends and workdays 
fridays = []
saturdays = []
sundays = []
workdays = []

dates = list(filter(lambda x: "." in x, df.columns))
for weekday in dates:
    daynum = datetime.strptime(weekday, '%d.%m.%y').isoweekday()
    if daynum == 5: fridays.append(weekday)
    elif daynum == 6: saturdays.append(weekday)
    elif daynum == 7: sundays.append(weekday)
    else: workdays.append(weekday)

In [ ]:
# split the period by 2 equal "dates" parts 
if len(dates)%2 == 0:
    index_2nd = len(dates)//2
    index_1st = 0
else:
    index_2nd = len(dates)//2+1
    index_1st = 1
    
first_half = dates[index_1st : index_2nd]
second_half = dates[index_2nd : len(dates)]
total_days = len(first_half) + len(second_half)


# retention & avr_check cals
df['first_half'] = df[first_half].sum(axis=1)
df['second_half'] = df[second_half].sum(axis=1)

def retention_calc(first_half, second_half):
    try:
        retention = second_half / first_half
    except:
        retention = 0
    retention = min (1, retention)
    return retention

df['retention'] = df.apply(lambda x: retention_calc(x['first_half'], x['second_half']), axis =  1)
df['total_rides'] = df['first_half'] + df['second_half']
df['avr_check'] = df['gmv'] / df['total_rides']
df['WK_rides'] = df[fridays + saturdays + sundays].sum(axis=1) # weekends rides
df['WD_rides'] = df[workdays].sum(axis=1) # weekends rides


key_fields = ['id', 'gmv','first_half','second_half','total_rides', 'WD_rides','WK_rides', 'avr_check', 'retention'] 
 
df = df[np.isfinite(df).all(1)] # infinity values drop (comes from a division by zero)
df = df.round({'gmv': 0, 'retention': 2, 'avr_check': 1})

## Результат расчета кол-ва поездок


**id** - id водителя<br>
**gmv** - gmv водителя<br>
**first_half** - количество поездок в первую половину периода<br>
**second_half** - количество поездок в первую половину периода<br>
**total_rides** - всего поездок за период
**WD_rides** - количество поездок в WorkDays<br>
**WK_rides** - количество поездок в WeeKends<br>
**avr_check** - средний чек<br>



In [ ]:
print('total drivers:', df.shape[0])
print('\nвыгрузка рандомных водителей')
df[key_fields].sample(n=5)

total drivers: 24905

выгрузка рандомных водителей


,id,gmv,first_half,second_half,total_rides,WD_rides,WK_rides,avr_check,retention
5294,36124653,10.0,0,3,3,3,0,3.3,1.00
14188,96572887,7.0,0,2,2,2,0,3.5,1.00
17786,21607807,332.0,45,43,88,35,58,3.8,0.96
4858,18296860,440.0,56,68,124,57,67,3.5,1.00
5168,80294053,20.0,0,6,6,6,0,3.3,1.00


## Расчитываем размеры когорт

In [ ]:
# exclude IDs which have zero rides during the first half of the period
df_activer_drivers = df.loc[df['first_half'] != 0]
    
all_cohorts = {}
range_start = 0

for percentile in percentile_level.keys():
    
    max_value = np.percentile(np.array(df_activer_drivers.total_rides.tolist()), percentile_level[percentile]) # percentile value
    all_cohorts[percentile] = range(int(range_start), int(max_value))
    range_start = max_value

last_percentile = int(max(list(percentile_level.keys())))+1
all_cohorts[last_percentile] = range(int(max_value),
                                     int(1e8)) # set a large number to keep the cohort range            

# cohort distribution for all IDs
def cohort_check(x, all_cohorts):
    for cohort in all_cohorts.keys():
        if x in all_cohorts[cohort]:
            return cohort
        else:
            pass      
df['cohort'] = df.apply(lambda x: cohort_check(x['total_rides'], all_cohorts), axis =  1)

In [ ]:
# shuffle data within cohorts
def shuffle(df,      # dataframe to shuffle data 
            cohorts): # cohorts to shuffle data within
    shuffle_df = pd.DataFrame()
    for cohort in set(cohorts):
        shuffle_df = shuffle_df.append(df[df.cohort == cohort].sample(frac=1))
    return shuffle_df

df = shuffle(df, df['cohort'].tolist())
df.reset_index(drop=True, inplace = True)
df.sample(n=5)

,id,gmv,29.05.22,30.05.22,31.05.22,01.06.22,02.06.22,03.06.22,04.06.22,05.06.22,...,25.06.22,26.06.22,first_half,second_half,retention,total_rides,avr_check,WK_rides,WD_rides,cohort
16857,35421159,154.0,0,0,0,0,0,0,0,0,...,1,0,0,44,1.00,44,3.5,23,21,4
6343,91870689,3.0,0,1,0,0,0,0,0,0,...,0,0,1,0,0.00,1,3.0,0,1,1
21040,82638812,391.0,8,11,6,8,2,4,3,8,...,0,0,70,33,0.47,103,3.8,52,59,5
18168,79104976,240.0,0,0,0,0,3,0,10,8,...,0,2,39,28,0.72,67,3.6,36,31,4
19274,20568773,254.0,0,1,1,4,3,0,2,0,...,4,4,34,38,1.00,72,3.5,29,43,4


<H3>cohorts features calc</H3>

In [ ]:
avr_check = {}
for cohort in range(1, cohorts+1):
    avr_check[cohort] = df['avr_check'].loc[df['cohort'] == cohort].mean()

avr_rides = {}
for cohort in range(1, cohorts+1):
    avr_rides[cohort] = df['total_rides'].loc[df['cohort'] == cohort].mean()
    
avr_wd_rides = {}
for cohort in range(1, cohorts+1):
    avr_wd_rides[cohort] = df['WD_rides'].loc[df['cohort'] == cohort].mean()
    
avr_wk_rides = {}
for cohort in range(1, cohorts+1):
    avr_wk_rides[cohort] = df['WK_rides'].loc[df['cohort'] == cohort].mean()

drivers = {}
for cohort in range(1, cohorts+1):
    drivers[cohort] = df['id'].loc[df['cohort'] == cohort].count()

avr_retention = {}
for cohort in range(1, cohorts+1):
    avr_retention[cohort] = df['retention'].loc[df['cohort'] == cohort].mean()

cohort_df = pd.DataFrame(list(avr_check.items()),
                   columns=['cohort', 'avr_check'])
cohort_df['drivers'] = list(drivers.values())
cohort_df['avr_rides'] = list(avr_rides.values())
cohort_df['WK_all_rides'] = list(avr_wk_rides.values()) # quantity of rides on weekends 
cohort_df['WD_all_rides'] = list(avr_wd_rides.values()) # quantity of rides on workdays 


# 'WK_avr_rides' - # avr. rides on a particular weekend
weekends_days = np.mean([len(fridays), len(saturdays), len(sundays)])
cohort_df['WK_avr_rides'] = cohort_df['WK_all_rides'] / weekends_days

# 'WD_avr_rides' - # avr. rides on a particular weekend
workdays_days = len(workdays)
cohort_df['WD_avr_rides'] = cohort_df['WD_all_rides'] / workdays_days

# retention calc
cohort_df = pd.merge(df.groupby('cohort')['retention'].mean(),
                     cohort_df,
                     on="cohort")

cohort_df['avr_rev_total'] = (cohort_df['avr_check'] * cohort_df['avr_rides'] * commission).astype(int)
cohort_df = cohort_df.round({'retention': 2,
                             'avr_check': 1,
                             'avr_rides': 1,
                             'WK_all_rides': 1,
                             'WD_all_rides': 1,
                             'WK_avr_rides': 1,
                             'WD_avr_rides': 1,
                            })

## Описание когорт 

In [ ]:
print('\ncohorts and range of rides:')
all_cohorts


cohorts and range of rides:


{1: range(0, 8),
 2: range(8, 21),
 3: range(21, 42),
 4: range(42, 82),
 5: range(82, 100000000)}

In [ ]:
print("описание когорт:")
cohort_df

описание когорт:


,cohort,retention,avr_check,drivers,avr_rides,WK_all_rides,WD_all_rides,WK_avr_rides,WD_avr_rides,avr_rev_total
0,1,0.60,3.8,6670,3.2,1.7,1.7,0.4,0.1,1
1,2,0.72,3.7,4996,13.3,7.0,6.8,1.6,0.4,4
2,3,0.78,3.7,4664,29.9,15.8,15.1,3.6,0.9,10
3,4,0.84,3.7,4377,58.9,30.9,29.8,7.1,1.9,20
4,5,0.88,3.7,4198,129.3,69.5,64.2,16.0,4.0,45


<h3>Target Rides and Incentive</h3>

In [ ]:
# bonus to be given as an inventive by groups, % from average check of the cohort  
# 1 equals 100% from the GMV of the target rides
# example: gmv = $10, 0.35 = $3.5 as the bonus

bonuses = {           
    1: 0.35,
    2: 0.35,
    3: 0.35,
    4: 0.35,
    5: 0}

## Расчитываем цели и вознаграждения

In [ ]:
# cohort_rides - maximum rides out of workdays & weekends rides
#cohort_df['cohort_rides'] = cohort_df[['WK_avr_rides', 'WD_avr_rides']].max(axis=1)

# rounding up all rides
cohort_df['cohort_rides'] = cohort_df['WK_avr_rides'].apply(lambda x: math.ceil(x))

# calc the target rides
new_rides = {}
for cohort in range(1, cohorts):
    new_rides[cohort] = cohort_df.loc[cohort,'cohort_rides'] - cohort_df.loc[cohort-1,'cohort_rides']

new_rides[cohorts] = 0 # set zero for the last upper cohort as there is no insentive

# 'WK_rides_extra' - Weekend Extra Rides to be done
cohort_df['WK_rides_extra'] = cohort_df['cohort'].map(new_rides) + 1
# add 1 additional ride to overachieve cohort border 
    
# 'rides_tbd' - all weekends rides to get the bonus
cohort_df['rides_tbd'] = cohort_df['cohort_rides'] + cohort_df['WK_rides_extra']
cohort_df.loc[cohorts-1,'rides_tbd'] = 0 # the target for the senior cohort shall be zero

# incentive for drivers
cohort_df['bonus, %'] = cohort_df['cohort'].map(bonuses)
cohort_df['driver_bonus'] = cohort_df['WK_rides_extra'] * cohort_df['avr_check'] * cohort_df['bonus, %']
cohort_df = cohort_df.round({'driver_bonus': 0})

In [ ]:
cohort_df

,cohort,retention,avr_check,drivers,avr_rides,WK_all_rides,WD_all_rides,WK_avr_rides,WD_avr_rides,avr_rev_total,...,"bonus, %",driver_bonus,win_rate,costs,ID_extra_rev,P: not FF: rev,P: FF: rides,P: FF: gmv,P: FF: rev,P: rev_up
0,1,0.60,3.8,6670,3.2,1.7,1.7,0.4,0.1,1,...,0.40,3.0,0.15,3001.50,722.361000,0.346560,6.640813,24,1,653.766720
1,2,0.72,3.7,4996,13.3,7.0,6.8,1.6,0.4,4,...,0.35,4.0,0.15,2997.60,790.242300,1.682982,14.966660,54,4,1736.373289
2,3,0.78,3.7,4664,29.9,15.8,15.1,3.6,0.9,10,...,0.35,6.0,0.15,4197.60,1229.547000,4.098841,29.451451,107,8,2729.250487
3,4,0.84,3.7,4377,58.9,30.9,29.8,7.1,1.9,20,...,0.30,10.0,0.15,8535.15,2076.995925,8.695407,64.674964,237,19,6765.480534
4,5,0.88,3.7,4198,129.3,69.5,64.2,16.0,4.0,45,...,0.00,0.0,0.15,0.00,221.339550,0.000000,0.000000,0,0,0.000000


## Результат расчетов

**WD_avr_rides**    - среднее кол-во поездок в workdays<br> 
**WK_avr_rides**    - среднее кол-во поездок в weekends<br>
**cohort_rides**    - среднее кол-во поездок в когорте<br>
**WK_rides_extra**  - на сколько больше надо сделать поездок в выходные<br>
**rides_tbd**       - сколько в итоге надо сделать поездок в выходые всего<br>
**driver_bonus**    - сколько бонусов предлагаем водителю за выполнение цели<br>
**avr_check**       - средний чек водителя (для сравнения)<br>

In [ ]:
cohort_df[['cohort',
           'WD_avr_rides',
           'WK_avr_rides',
           'cohort_rides',
           'WK_rides_extra',
           'rides_tbd',
           'driver_bonus',
           'avr_check']]

,cohort,WD_avr_rides,WK_avr_rides,cohort_rides,WK_rides_extra,rides_tbd,driver_bonus,avr_check
0,1,0.1,0.4,1,2,3,3.0,3.8
1,2,0.4,1.6,2,3,5,4.0,3.7
2,3,0.9,3.6,4,5,9,6.0,3.7
3,4,1.9,7.1,8,9,17,10.0,3.7
4,5,4.0,16.0,16,1,0,0.0,3.7


## Расчитываем окупаемость

<h3>Instant revenue and costs</h3>

In [ ]:
# instant revenue
cohort_df['win_rate'] = win_rate

# costs
cohort_df['costs'] = cohort_df['driver_bonus'] * \
    cohort_df['drivers'] * win_rate

# extra revenue for the company
cohort_df['ID_extra_rev'] = cohort_df['WK_rides_extra'] \
    * cohort_df['avr_check'] \
    * cohort_df['drivers'] \
    * win_rate \
    * commission

instant_rev = cohort_df['ID_extra_rev'].sum()
instant_cost = cohort_df['costs'].sum()

print("instant revenue:", int(instant_rev))
print("instant costs  :", int(instant_cost))
print("instant balance:", int(instant_rev - instant_cost))

instant revenue: 5040
instant costs  : 16762
instant balance: -11721


<h2>Perspective revenue projection</h2>

<h3>Target is not fullfilled</h3>

In [ ]:
# target IS NOT fulfilled
# 'P: not FF: rev' - Perspective Not Fulfilled: Revenue

cohort_df['P: not FF: rev'] = cohort_df['avr_rides'] \
                                * cohort_df['avr_check'] \
                                * cohort_df['retention'] \
                                * period_duration \
                                * commission
cohort_df.loc[cohorts-1,'P: not FF: rev'] = 0 #keep the last cohort zero as no an insentive for the cohort

<h3>Target is fullfilled</h3>

In [ ]:

# target IS fulfilled
FF_rides = {}
for i in range (1, cohorts):
    FF_rides[i] = avr_rides[i+1] * period_duration
FF_rides[cohorts] = 0
cohort_df['P: FF: rides'] = list(FF_rides.values())

FF_gmv = {}
for i in range (1, cohorts):
    FF_gmv[i] = int(FF_rides[i] * avr_check[i+1])
FF_gmv[cohorts] = 0
cohort_df['P: FF: gmv'] = list(FF_gmv.values())

FF_rev = {}
for i in range (1, cohorts):
    FF_rev[i] = int(FF_gmv[i] * commission * avr_retention[i+1])
FF_rev[cohorts] = 0
cohort_df['P: FF: rev'] = list(FF_rev.values())

cohort_df['P: rev_up'] = (cohort_df['P: FF: rev'] - cohort_df['P: not FF: rev']) \
                                * win_rate \
                                * cohort_df['drivers']

persp_rev_up = (cohort_df['P: rev_up'].sum())
revenue_total = cohort_df['ID_extra_rev'].sum() + persp_rev_up

In [ ]:
cohort_df

,cohort,retention,avr_check,drivers,avr_rides,WK_all_rides,WD_all_rides,WK_avr_rides,WD_avr_rides,avr_rev_total,...,"bonus, %",driver_bonus,win_rate,costs,ID_extra_rev,P: not FF: rev,P: FF: rides,P: FF: gmv,P: FF: rev,P: rev_up
0,1,0.60,3.8,6670,3.2,1.7,1.7,0.4,0.1,1,...,0.40,3.0,0.15,3001.5,722.361000,0.346560,6.640813,24,1,653.766720
1,2,0.72,3.7,4996,13.3,7.0,6.8,1.6,0.4,4,...,0.35,4.0,0.15,2997.6,790.242300,1.682982,14.966660,54,4,1736.373289
2,3,0.78,3.7,4664,29.9,15.8,15.1,3.6,0.9,10,...,0.35,6.0,0.15,4197.6,1229.547000,4.098841,29.451451,107,8,2729.250487
3,4,0.84,3.7,4377,58.9,30.9,29.8,7.1,1.9,20,...,0.30,10.0,0.15,6565.5,2076.995925,8.695407,64.674964,237,19,6765.480534
4,5,0.88,3.7,4198,129.3,69.5,64.2,16.0,4.0,45,...,0.00,0.0,0.15,0.0,221.339550,0.000000,0.000000,0,0,0.000000


## Результат расчета

In [ ]:
print('instant revenue      :', int(instant_rev))
print('perspective revenue  :', int(persp_rev_up))
print('revenue total        :', int(revenue_total))
print('costs:               :', int(instant_cost))
print('profit:              :', int(revenue_total - instant_cost))

instant revenue      : 5040
perspective revenue  : 11884
revenue total        : 16925
costs:               : 16762
profit:              : 163


## Распределяем водителей по группам для тестов разного размера бонусов

распределяем водителей по группа для теста: конрольная и 3 тестовые

In [ ]:
group_split = {
    "control": 0.25, # 25% of drivers go the control group
    "a": 0.25,
    "b": 0.25,
    "c": 0.25}

if sum(group_split.values()) != 1: print("split in test/control groups is NOT correct") 

распределяем долю бонусов от расчитанных, которые мы отдаем до тестов<br>
**control** - не даем бонусы<br>
**остальные** - 1: отдаем 100% расчитанных бонусов, 0.5% отдаем 50% и т.д.

In [ ]:
bonuses_split = {
    "control": 0,
    "a": 1,       # 100% of target bonuses will be given to drivers
    "b": 0.8,     # 
    "c": 0.6}     # 

## Разбиваем водителей группы для теста

In [ ]:
df_groups = pd.DataFrame(columns = ['id', 'groups'])
for cohort in range (1, cohorts+1):
    group_length = df[df['cohort'] == cohort].shape[0]
    control = ['control' for i in range(1, int(group_length * group_split['control']))]
    a = ['a' for i in range(1, int(group_length * group_split['a']))]
    b = ['b' for i in range(1, int(group_length * group_split['b']))]
    c = ['c' for i in range (1, group_length - len(control) - len(a) - len(b) + 1)]

    df_group = df[df['cohort'] == cohort]
    df_group['groups'] = control + a + b + c
    df_group = df_group[['id', 'groups']]
    df_groups = pd.concat([df_groups, df_group])
    
df = df.merge(df_groups, how = 'left', on = 'id')

result_df = df.merge(cohort_df, how='inner', on='cohort')[['id','cohort','driver_bonus','groups', 'rides_tbd']]

result_df['groups_coeff'] = result_df['groups'].apply(lambda x: bonuses_split[x])
result_df['bonus_to_offer'] = result_df['groups_coeff'] * result_df['driver_bonus']
result_df['bonus_to_offer'] = result_df['bonus_to_offer'].apply(lambda x: round(x,0))
result_df.sample(5)

,id,cohort,driver_bonus,groups,rides_tbd,groups_coeff,bonus_to_offer
1685,58545897,1,3.0,a,3,1.0,3.0
10723,36679490,2,4.0,c,5,0.6,2.0
21798,41845536,5,0.0,a,0,1.0,0.0
936,123183240,1,3.0,control,3,0.0,0.0
12731,17483894,3,6.0,control,9,0.0,0.0


## Смотрим на распределение водителеям по тестовым группам

In [ ]:
print("РАЗБИВКА ВОДИТЕЛЕЙ ПО ГРУППАМ:")

print("\ntotal drivers:", df.shape[0])
for cohort in range (1, cohorts + 1):
    print(" cohort", cohort, ":", df[df['cohort'] == cohort].shape[0])
    for group in list(group_split.keys()):
        print("  ", group, ":", df[(df['cohort'] == cohort) & (df['groups'] == group)].shape[0])

print('\nПРИМЕР РАЗБИВКИ ПО КОГОРТАМ')
for cohort in range(1, cohorts):
    print('\ncohort:', cohort)
    print(result_df[result_df.cohort == cohort].groupby(['groups']).mean())

РАЗБИВКА ВОДИТЕЛЕЙ ПО ГРУППАМ:

total drivers: 24905
 cohort 1 : 6670
   control : 1666
   a : 1666
   b : 1666
   c : 1672
 cohort 2 : 4996
   control : 1248
   a : 1248
   b : 1248
   c : 1252
 cohort 3 : 4664
   control : 1165
   a : 1165
   b : 1165
   c : 1169
 cohort 4 : 4377
   control : 1093
   a : 1093
   b : 1093
   c : 1098
 cohort 5 : 4198
   control : 1048
   a : 1048
   b : 1048
   c : 1054

ПРИМЕР РАЗБИВКИ ПО КОГОРТАМ

cohort: 1
         cohort  driver_bonus  rides_tbd  groups_coeff  bonus_to_offer
groups                                                                
a           1.0           3.0        3.0           1.0             3.0
b           1.0           3.0        3.0           0.8             2.0
c           1.0           3.0        3.0           0.6             2.0
control     1.0           3.0        3.0           0.0             0.0

cohort: 2
         cohort  driver_bonus  rides_tbd  groups_coeff  bonus_to_offer
groups                                      

## Сохранаяем расчеты в файл

название файл будет в формате "bonus_table_день_месяц_час_минуты"

In [ ]:
file_name = "bonus_table_" + datetime.today().strftime('%d%m_%H%M') + '.csv'
result_df.to_csv(file_name, index=False)